## 1980 年至 2023 年的热门电子游戏数据

games.csv 
简介：数据集包含从 1980 年到 2023 年的视频游戏列表，并提供发布日期、用户评论评级和评论家评论评级等信息。


## 分析目标

本次分析的目的是为了整合不同游戏流派中，各游戏的平均评分，从而挖掘出各个流派中的高分游戏和各个团队的高分游戏

变量含义：
- Title：游戏标题
- Release Date：游戏首个版本发布日期
- Team：游戏开发团队
- Rating：平均评分
- Times Listed：列出此游戏的用户数量
- Number of Reviews：用户提供的评论数量
- Genres：游戏所属的所有类型/流派
- Summary：团队提供的摘要/概述
- Reviews：用户的评价/评论

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

### 读取数据

In [2]:
orignial_game_data = pd.read_csv('games.csv')

In [3]:
orignial_game_data

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K
1,1,Hades,"Dec 10, 2019",['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,21K,3.2K,6.3K,3.6K
2,2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,30K,2.5K,5K,2.6K
3,3,Undertale,"Sep 15, 2015","['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,28K,679,4.9K,1.8K
4,4,Hollow Knight,"Feb 24, 2017",['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",21K,2.4K,8.3K,2.3K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,1507,Back to the Future: The Game,"Dec 22, 2010",['Telltale Games'],3.2,94,94,"['Adventure', 'Point-and-Click']",Back to the Future: The Game is one of Telltal...,['Very enjoyable game. The story adds onto the...,763,5,223,67
1508,1508,Team Sonic Racing,"May 21, 2019","['Sumo Digital', 'Sega']",2.9,264,264,"['Arcade', 'Racing']",Team Sonic Racing combines the best elements o...,"['jogo morto mas bom', 'not my cup of tea', ""C...",1.5K,49,413,107
1509,1509,Dragon's Dogma,"May 22, 2012",['Capcom'],3.7,210,210,"['Brawler', 'RPG']","Set in a huge open world, Dragon’s Dogma: Dark...","['Underrated.', 'A grandes rasgos, es como un ...",1.1K,45,487,206
1510,1510,Baldur's Gate 3,"Oct 06, 2020",['Larian Studios'],4.1,165,165,"['Adventure', 'RPG', 'Strategy', 'Tactical', '...","An ancient evil has returned to Baldur's Gate,...",['Bu türe bu oyunla girmeye çalışmak hataydı s...,269,79,388,602


In [4]:
cleaned_game_data = orignial_game_data.copy()

### 评估数据

In [5]:
cleaned_game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1512 non-null   int64  
 1   Title              1512 non-null   object 
 2   Release Date       1512 non-null   object 
 3   Team               1511 non-null   object 
 4   Rating             1499 non-null   float64
 5   Times Listed       1512 non-null   object 
 6   Number of Reviews  1512 non-null   object 
 7   Genres             1512 non-null   object 
 8   Summary            1511 non-null   object 
 9   Reviews            1512 non-null   object 
 10  Plays              1512 non-null   object 
 11  Playing            1512 non-null   object 
 12  Backlogs           1512 non-null   object 
 13  Wishlist           1512 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 165.5+ KB


#### 评估数据整齐度

数据的结构性问题指不符合“每个变量为一列，每个观察值为一行，每种类型的观察单位为一个表格”这三个标准；数据的内容性问题包括存在丢失数据、重复数据、无效数据等。

很明显，games表中`Team`和`Genres`列中不满足“每个变量为一列”标准，因此需要进行分割。并且`Rating`,`Team`,`Summary`列中存在缺失值，后续对其进行检查。`Release Date`的数据类型应该为datetime类型。

Summary和Reviews对此次分析并无意义，应删除

In [6]:
cleaned_game_data.drop(['Summary','Reviews'], axis = 1, inplace = True)

In [7]:
print(type(cleaned_game_data['Team'][1]))
print(type(cleaned_game_data['Genres'][1]))

<class 'str'>
<class 'str'>


在info中Team是存在缺失值的，但是该缺失值对后续分析没有影响，且数据行还包含了其他重要数据，故可保留此缺失值

aval函数不可包含缺失值，所以需要对缺失值进行处理，

In [8]:
cleaned_game_data.query('Team.isna()')

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Plays,Playing,Backlogs,Wishlist
1245,1245,NEET Girl Date Night,"Oct 21, 2022",NaN,2.7,21,21,['Visual Novel'],106,1,44,42


In [9]:
cleaned_game_data['Team'] = cleaned_game_data['Team'].fillna('[]')

In [10]:
cleaned_game_data['Team'] = cleaned_game_data['Team'].apply(lambda x: eval(x))
cleaned_game_data['Genres'] = cleaned_game_data['Genres'].apply(lambda x: eval(x))

In [11]:
cleaned_game_data.sample(10)

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Plays,Playing,Backlogs,Wishlist
1365,1365,Crash Team Racing Nitro-Fueled,"Jun 21, 2019","[Activision Blizzard, Beenox]",3.9,408,408,"[Arcade, Racing]",2.5K,50,391,294
579,579,NieR,"Apr 22, 2010","[Square Enix, Cavia]",4.2,743,743,"[Adventure, RPG, Shooter, Simulator, Strategy]",2.9K,67,999,854
991,991,Metal Gear Solid 3: Subsistence,"Dec 22, 2005","[Konami Computer Entertainment Japan, Konami]",4.6,641,641,"[Adventure, Shooter, Tactical]",3.7K,50,949,639
796,796,Pokémon Legends: Arceus,"Jan 28, 2022","[Nintendo, Game Freak]",3.7,1.6K,1.6K,"[Adventure, RPG, Turn Based Strategy]",9.1K,1.6K,2.5K,2.1K
343,343,Red Dead Redemption 2,"Oct 26, 2018","[Take-Two Interactive, Rockstar Games]",4.4,2.9K,2.9K,"[Adventure, RPG, Shooter]",19K,1.7K,5.5K,2.9K
1325,1325,Puyo Puyo Tetris,"Feb 06, 2014","[Sega, Sonic Team]",3.8,233,233,"[Puzzle, Strategy]",3.1K,51,333,198
877,877,Borderlands 2,"Sep 18, 2012","[Gearbox Software, 2K Games]",3.5,1.1K,1.1K,"[RPG, Shooter]",16K,344,2.7K,494
839,839,Dark Souls,"Sep 22, 2011","[FromSoftware, Bandai Namco Entertainment]",4.3,2.1K,2.1K,"[Adventure, RPG]",14K,447,2.5K,1.4K
168,168,Pokémon Platinum,"Sep 13, 2008","[Nintendo, Game Freak]",4.1,1.4K,1.4K,"[Adventure, RPG, Turn Based Strategy]",14K,254,1.3K,461
773,773,Gang Beasts,"Dec 12, 2017","[Boneloaf, Double Fine Productions]",3.1,207,207,"[Fighting, Indie]",3.2K,30,177,142


In [12]:
cleaned_game_data = cleaned_game_data.explode('Team')
cleaned_game_data = cleaned_game_data.explode('Genres')

In [13]:
cleaned_game_data.head()

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022",Bandai Namco Entertainment,4.5,3.9K,3.9K,Adventure,17K,3.8K,4.6K,4.8K
0,0,Elden Ring,"Feb 25, 2022",Bandai Namco Entertainment,4.5,3.9K,3.9K,RPG,17K,3.8K,4.6K,4.8K
0,0,Elden Ring,"Feb 25, 2022",FromSoftware,4.5,3.9K,3.9K,Adventure,17K,3.8K,4.6K,4.8K
0,0,Elden Ring,"Feb 25, 2022",FromSoftware,4.5,3.9K,3.9K,RPG,17K,3.8K,4.6K,4.8K
1,1,Hades,"Dec 10, 2019",Supergiant Games,4.3,2.9K,2.9K,Adventure,21K,3.2K,6.3K,3.6K


In [14]:
cleaned_game_data.drop('Unnamed: 0',axis = 1, inplace = True)

In [15]:
cleaned_game_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6489 entries, 0 to 1511
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              6489 non-null   object 
 1   Release Date       6489 non-null   object 
 2   Team               6488 non-null   object 
 3   Rating             6457 non-null   float64
 4   Times Listed       6489 non-null   object 
 5   Number of Reviews  6489 non-null   object 
 6   Genres             6485 non-null   object 
 7   Plays              6489 non-null   object 
 8   Playing            6489 non-null   object 
 9   Backlogs           6489 non-null   object 
 10  Wishlist           6489 non-null   object 
dtypes: float64(1), object(10)
memory usage: 608.3+ KB


Team的缺失值已经处理完毕，接下来处理Rating的缺失值

In [16]:
cleaned_game_data[cleaned_game_data['Rating'].isna()]

,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Plays,Playing,Backlogs,Wishlist
587,Final Fantasy XVI,"Jun 22, 2023",Square Enix,NaN,422,422,RPG,37,10,732,2.4K
587,Final Fantasy XVI,"Jun 22, 2023",Square Enix Creative Business Unit III,NaN,422,422,RPG,37,10,732,2.4K
649,Death Stranding 2,releases on TBD,Kojima Productions,NaN,105,105,Adventure,3,0,209,644
649,Death Stranding 2,releases on TBD,Kojima Productions,NaN,105,105,Shooter,3,0,209,644
713,Final Fantasy VII Rebirth,"Dec 31, 2023",Square Enix,NaN,192,192,NaN,20,3,354,1.1K
719,Lies of P,"Aug 01, 2023",NEOWIZ,NaN,175,175,RPG,5,0,260,939
719,Lies of P,"Aug 01, 2023",Round8 Studio,NaN,175,175,RPG,5,0,260,939
726,Judas,"Mar 31, 2025",Ghost Story Games,NaN,90,90,Adventure,1,0,92,437
726,Judas,"Mar 31, 2025",Ghost Story Games,NaN,90,90,Shooter,1,0,92,437
746,Like a Dragon Gaiden: The Man Who Erased His Name,"Dec 31, 2023",Ryū Ga Gotoku Studios,NaN,118,118,Adventure,2,1,145,588


以上数据缺失了分析所需的核心数据Rating，我们将把这些观察值删除。

In [17]:
cleaned_game_data.dropna(subset = ['Rating'], inplace = True)
cleaned_game_data['Rating'].isna().sum()

0

Rating的缺失值已经处理完毕.

In [19]:
cleaned_game_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6457 entries, 0 to 1511
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              6457 non-null   object 
 1   Release Date       6457 non-null   object 
 2   Team               6456 non-null   object 
 3   Rating             6457 non-null   float64
 4   Times Listed       6457 non-null   object 
 5   Number of Reviews  6457 non-null   object 
 6   Genres             6454 non-null   object 
 7   Plays              6457 non-null   object 
 8   Playing            6457 non-null   object 
 9   Backlogs           6457 non-null   object 
 10  Wishlist           6457 non-null   object 
dtypes: float64(1), object(10)
memory usage: 605.3+ KB


### 处理重复数据

根据变量含义可知，本次所有变量允许存在重复数据，所以不需要进行重复数据处理

### 处理不一致数据

不一致数据可能存在于流派中，我们将查看是由存在多个不同值指代同一个流派

In [20]:
cleaned_game_data['Genres'].value_counts()

Genres
Adventure              1825
RPG                     928
Shooter                 654
Platform                597
Indie                   459
Puzzle                  307
Brawler                 292
Strategy                244
Simulator               209
Turn Based Strategy     184
Fighting                136
Visual Novel            120
Arcade                  118
Tactical                 87
Point-and-Click          81
Racing                   73
Sport                    43
Music                    41
Card & Board Game        25
Real Time Strategy       17
MOBA                      6
Quiz/Trivia               6
Pinball                   2
Name: count, dtype: int64

暂未发现有不一致的数据

### 处理无效或错误数据

In [21]:
cleaned_game_data.describe()

,Rating
count,6457.000000
mean,3.737618
std,0.524826
min,0.700000
25%,3.500000
50%,3.800000
75%,4.100000
max,4.800000


并不存在脱离现实的值

In [ ]:
cleaned_game_data